# 1. Importing Libraries

In [1]:
import lifetimes
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter # BG/NBD
from lifetimes import GammaGammaFitter # Gamma-Gamma Model
from lifetimes.plotting import plot_frequency_recency_matrix
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


https://medium.com/@ugursavci/customer-lifetime-value-prediction-in-python-89e4a50df12e

# 2. Reading Data

In [2]:
df = pd.read_csv('data/sample.csv')
df.head()

,invoice,date,customer_id,stockcode,price
0,1,2020-07-25,d18734,y7,50.45
1,2,2020-01-17,c21086,x7,25.30
2,3,2019-07-05,d18185,z5,18.40
3,4,2019-02-26,c18331,z2,5.50
4,5,2019-02-10,b16309,y7,18.40


# 3. Understanding Data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   invoice      100000 non-null  int64  
 1   date         100000 non-null  object 
 2   customer_id  100000 non-null  object 
 3   stockcode    100000 non-null  object 
 4   price        100000 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 3.8+ MB


In [4]:
df.describe()

,invoice,price
count,100000.000000,100000.000000
mean,50000.500000,22.618414
std,28867.657797,15.425024
min,1.000000,5.500000
25%,25000.750000,12.990000
50%,50000.500000,18.400000
75%,75000.250000,25.300000
max,100000.000000,50.450000


# 4. Data Preprocessing

## Filtering Our Data

In [7]:
# df = df[df['Quantity'] > 0 ] # exclude the orders with 0 value
# df = df[df['UnitPrice'] > 0] # exclude the Unit Price with 0 value
# df = df[~df['InvoiceNo'].str.contains("C",na=False)]  # C indicates the returned orders we don't want them as well

## Missing Values

In [8]:
df.isnull().sum()

invoice        0
date           0
customer_id    0
stockcode      0
price          0
dtype: int64

In [9]:
df.dropna(inplace=True)  # inplace=True means we dropped them permanently

## Handling Outliers

In [10]:
def find_boundaries(df, variable,q1=0.05,q2=0.95):
    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1) # lower quantile
    upper_boundary = df[variable].quantile(q2) # upper quantile
    return upper_boundary, lower_boundary
def capping_outliers(df,variable):
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                            np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

In [16]:
# capping_outliers(df,'price')
# capping_outliers(df,'stockcode')

In [17]:
df.describe()

,invoice,price
count,100000.000000,100000.000000
mean,50000.500000,22.618414
std,28867.657797,15.425024
min,1.000000,5.500000
25%,25000.750000,12.990000
50%,50000.500000,18.400000
75%,75000.250000,25.300000
max,100000.000000,50.450000


## Preparing Our Dataset ( RFM Dataset )

In [18]:
# df['Total Price'] = df['UnitPrice'] * df['Quantity']

In [19]:
df.head()

,invoice,date,customer_id,stockcode,price
0,1,2020-07-25,d18734,y7,50.45
1,2,2020-01-17,c21086,x7,25.30
2,3,2019-07-05,d18185,z5,18.40
3,4,2019-02-26,c18331,z2,5.50
4,5,2019-02-10,b16309,y7,18.40


In [20]:
df['date'].max()

'2020-12-31'

In [21]:
clv = lifetimes.utils.summary_data_from_transaction_data(df,'customer_id','date','price',observation_period_end='2020-12-31')
clv.head()

,frequency,recency,T,monetary_value
customer_id,,,,
a15600,2.0,501.0,653.0,25.3000
a15601,1.0,230.0,399.0,5.5000
a15602,2.0,339.0,655.0,27.9750
a15603,2.0,253.0,274.0,19.1450
a15604,4.0,570.0,576.0,18.7725


In [22]:
clv = clv[clv['frequency']>1] # we want only customers shopped more than 2 times
clv.head()

,frequency,recency,T,monetary_value
customer_id,,,,
a15600,2.0,501.0,653.0,25.3000
a15602,2.0,339.0,655.0,27.9750
a15603,2.0,253.0,274.0,19.1450
a15604,4.0,570.0,576.0,18.7725
a15605,4.0,530.0,682.0,26.7850


## Frequency/Recency analysis using the BG/NBD model

In [23]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(clv['frequency'], clv['recency'], clv['T'])

<lifetimes.BetaGeoFitter: fitted with 19021 subjects, a: 0.00, alpha: 1301.56, b: 0.00, r: 6.96>

## Expected Number of Purchases within 6 Months

In [24]:
t = 180 # 30 day period
clv['expected_purc_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, clv['frequency'], clv['recency'], clv['T'])
clv.sort_values(by='expected_purc_6_months',ascending=False).head(5)

,frequency,recency,T,monetary_value,expected_purc_6_months
customer_id,,,,,
a17404,10.0,529.0,561.0,17.28400,1.639356
e20570,8.0,307.0,348.0,29.18625,1.632799
b18542,10.0,555.0,596.0,22.46900,1.609119
d17760,10.0,577.0,619.0,24.95900,1.589848
b19580,10.0,643.0,652.0,18.03300,1.562992


## Gamma-Gamma Model

In [25]:
clv[['frequency','monetary_value']].corr()

,frequency,monetary_value
frequency,1.000000,-0.007872
monetary_value,-0.007872,1.000000


In [26]:
clv[['frequency','monetary_value']].corr()

,frequency,monetary_value
frequency,1.000000,-0.007872
monetary_value,-0.007872,1.000000


In [28]:
# df[['UnitPrice','Quantity']].corr()

In [29]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(clv["frequency"],
        clv["monetary_value"])

<lifetimes.GammaGammaFitter: fitted with 19021 subjects, p: 4.25, q: 1.25, v: 4.02>

In [30]:
clv['6_monhths_clv']=ggf.customer_lifetime_value(bgf,
                                   clv["frequency"],
                                   clv["recency"],
                                   clv["T"],
                                   clv["monetary_value"],
                                   time=6,
                                   freq='D',
                                   discount_rate=0.01)

In [31]:
clv.sort_values('6_monhths_clv',ascending=False).head()

,frequency,recency,T,monetary_value,expected_purc_6_months,6_monhths_clv
customer_id,,,,,,
a19759,3.0,194.0,451.0,58.883333,1.023301,58.393584
a17859,2.0,102.0,133.0,50.450000,1.124664,55.378122
e22612,2.0,96.0,144.0,50.450000,1.116105,54.956720
c20119,5.0,428.0,429.0,45.420000,1.244336,54.918728
d17436,2.0,74.0,159.0,50.450000,1.104643,54.392311


In [32]:
clv['Segment'] =  pd.qcut(clv['6_monhths_clv'], 4, labels = ['Hibernating','Need Attention','LoyalCustomers', 'Champions'])

In [33]:
clv.head()

,frequency,recency,T,monetary_value,expected_purc_6_months,6_monhths_clv,Segment
customer_id,,,,,,,
a15600,2.0,501.0,653.0,25.3000,0.825452,21.159845,Need Attention
a15602,2.0,339.0,655.0,27.9750,0.824608,23.208578,LoyalCustomers
a15603,2.0,253.0,274.0,19.1450,1.024015,20.334105,Need Attention
a15604,4.0,570.0,576.0,18.7725,1.051043,19.791113,Need Attention
a15605,4.0,530.0,682.0,26.7850,0.994876,26.322737,LoyalCustomers


In [34]:
clv.groupby('Segment').mean()

,frequency,recency,T,monetary_value,expected_purc_6_months,6_monhths_clv
Segment,,,,,,
Hibernating,2.574853,423.270395,592.336627,12.017998,0.909326,11.688219
Need Attention,3.057834,430.381493,581.152892,18.513112,0.960641,17.964337
LoyalCustomers,3.333754,451.969295,596.535857,25.618478,0.978964,24.621176
Champions,3.319453,417.101577,555.081809,34.694028,0.999545,33.633532
